# <span style="color:green">Загрузка данных</span>

## 1. Импорт библиотек

In [1]:
import pandas as pd

## 2. Загрузка данных

In [8]:
try:
    data_sql = pd.read_excel("/Users/zigalovartur/Desktop/data_mts.xlsx" , sheet_name = "SQL")
    data = pd.read_excel("/Users/zigalovartur/Desktop/data_mts.xlsx" , sheet_name = "PPT")
    installs = pd.read_csv('/Users/angelinazigalova/Desktop/DEF-9xx.csv')
except:
    print('Не удалось открыть файлы')

Не удалось открыть файлы


# <span style="color:blue">Задание 1 (SQL)</span>

В таблице ACTV хранятся данные всех звонков абонентов за отчетный месяц с полем, 

указывающим на факт нахождения абонента в роуминге во время звонка: 
- 1 если абонент был в роуминге,
- 0 если не был в роуминге.

С помощью таблицы ACTV, вывести ID пять самых активных абонентов по количеству 

звонков в роуминге на каждый день.

In [3]:
data_sql

,ID_abon,Date of call,Time of call,Roaming
0,1,2017-01-01,13:01:00,1
1,1,2017-01-02,15:01:00,0
2,2,2017-01-01,09:01:00,0


WITH new AS (
    SELECT ID_abon,
           Date of call,
           ROW_NUMBER() OVER (PARTITION BY Date of call ORDER BY COUNT(CASE WHEN Roaming = 1 
                                                                            THEN 1 
                                                                            ELSE NULL 
                                                                            END) DESC) AS count
    FROM ACTV
    GROUP BY ID_abon, Date of call
)
SELECT ID_abon, 
       Date of call
FROM new
WHERE count <= 5

# <span style="color:blue">Задание 2</span>

Второе задание заключается в том, чтобы отрисовать слайд в 

PowerPoint (диаграммы на твой выбор), проанализировать данные.

Источником данных на листе PPT является выборка по звонкам 

абонентов с общей длительностью и количеством вход/исход звонков на абонента.

Для полного анализа сначала необходимо подтянуть оператора к каждому номеру. 

Емкости1 операторов можно взять на сайте Минцифры 
(Выписка по диапазону 9xx https://opendata.digital.gov.ru/registry/numeric/downloads ).

После отрисовать слайд в ppt в динамике/статике по натуральным и производным показателям. 

Проанализировать полученные данные, описать выводы и, если есть, ошибки в данных.

Прислать заполненный excel-файл с подтянутыми операторами (с формулами) и слайд в ppt.

1 Каждый оператор закупает для последующей реализации номерной диапазон.  

Пример диапазона: МТС принадлежат номера с 79160000000 по 79169999999 в Москве и МО.

# <span style="color:green">Предобработка данных</span>

In [4]:
data.head(3)

,Месяц,Моб_номер,Исход_голос_трафик_сек,Вход_голос_трафик_сек,Исход_голос_трафик_колво_звонков,Вход_голос_трафик_колво_звонков
0,2020-11-01,79263036XXX,736.8,1440.0,6,18
1,2020-01-01,79295212XXX,0.0,816.0,0,6
2,2020-12-01,79191093XXX,11340.8,11006.8,73,78


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11232 entries, 0 to 11231
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Месяц                             11232 non-null  datetime64[ns]
 1   Моб_номер                         11232 non-null  object        
 2   Исход_голос_трафик_сек            11232 non-null  float64       
 3   Вход_голос_трафик_сек             11232 non-null  float64       
 4   Исход_голос_трафик_колво_звонков  11232 non-null  int64         
 5   Вход_голос_трафик_колво_звонков   11232 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 526.6+ KB


<span style="color:orange">При первичном знакомстве с данными можно сказать:</span>

<span style="color:orange">1. Пропусков в данных нет</span>  

<span style="color:orange">2. Типы полей являются корректными</span>

<span style="color:orange">3. Для удобного анализа можно исправить названия полей - привести к змеиному регистру и перевести на латиницу</span>

## 1. Привожу названия полей к удобному виду

In [6]:
data.columns = ['month', 'phone_number', 'outgoing_sec', 'incoming_sec', 'outgoing_count', 'incoming_count']

In [7]:
data

,month,phone_number,outgoing_sec,incoming_sec,outgoing_count,incoming_count
0,2020-11-01,79263036XXX,736.8,1440.0,6,18
1,2020-01-01,79295212XXX,0.0,816.0,0,6
2,2020-12-01,79191093XXX,11340.8,11006.8,73,78
3,2020-05-01,79175256XXX,216.0,10316.4,4,68
4,2020-07-01,79778842XXX,24.0,0.0,1,0
...,...,...,...,...,...,...
11227,2020-06-01,79151195XXX,2744.4,648.0,17,5
11228,2020-08-01,79032680XXX,2822.4,1730.4,4,2
11229,2020-10-01,79161015XXX,2145.6,442.8,17,4
11230,2020-09-01,79091560XXX,8186.4,8137.2,78,60
